In [1]:
#Dependencies
import gmaps
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from scipy.stats import linregress
import json

# Google developer API key
from config import gkey

# Access maps with unique API key
gmaps.configure(api_key=gkey)

ImportError: DLL load failed: The specified module could not be found.

In [2]:
# The path to our bus stop locations CSV file
locations_file = "Resources/stops.csv"

# Read our file data into pandas
locations_df = pd.read_csv(locations_file)
locations_df.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
0,1.0,NaN,COLUMBUS ST & BROADWAY,NaN,41.392985,-81.536518,NaN,NaN,0,NaN
1,2.0,NaN,10800 BROOKPARK RD (HOME DEPOT),NaN,41.418571,-81.760947,NaN,NaN,0,NaN
2,4.0,NaN,EUCLID AV & E 100TH ST,NaN,41.503697,-81.618108,NaN,NaN,0,NaN
3,5.0,NaN,10081 W RIDGEWOOD DR,NaN,41.384506,-81.756317,NaN,NaN,0,NaN
4,14.0,NaN,PROSPECT AV & ONTARIO ST,NaN,41.498030,-81.692012,NaN,NaN,0,NaN


In [3]:
# The path to our bus stop times CSV file
stops_file = "Resources/stop_times.csv"

# Read our file data into pandas
stops_df = pd.read_csv(stops_file)
stops_df.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type
0,16741667,3:21:00 AM,3:21:00 AM,14507,1,0,0
1,16741667,3:22:00 AM,3:22:00 AM,4568,2,0,0
2,16741667,3:23:00 AM,3:23:00 AM,4522,3,0,0
3,16741667,3:24:00 AM,3:24:00 AM,4378,4,0,0
4,16741667,3:25:00 AM,3:25:00 AM,4556,5,0,0


In [4]:
# merge
bus_data = pd.merge(locations_df, stops_df, on=["stop_id", "stop_id"])
bus_data.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,trip_id,arrival_time,departure_time,stop_sequence,pickup_type,drop_off_type
0,1.0,NaN,COLUMBUS ST & BROADWAY,NaN,41.392985,-81.536518,NaN,NaN,0,NaN,16741990,5:54:00 AM,5:54:00 AM,82,0,0
1,1.0,NaN,COLUMBUS ST & BROADWAY,NaN,41.392985,-81.536518,NaN,NaN,0,NaN,16741993,6:54:00 AM,6:54:00 AM,82,0,0
2,1.0,NaN,COLUMBUS ST & BROADWAY,NaN,41.392985,-81.536518,NaN,NaN,0,NaN,16741995,7:25:00 AM,7:25:00 AM,82,0,0
3,1.0,NaN,COLUMBUS ST & BROADWAY,NaN,41.392985,-81.536518,NaN,NaN,0,NaN,16741997,7:57:00 AM,7:57:00 AM,82,0,0
4,1.0,NaN,COLUMBUS ST & BROADWAY,NaN,41.392985,-81.536518,NaN,NaN,0,NaN,16741999,8:28:00 AM,8:28:00 AM,82,0,0


In [5]:
#groupby stop id to see how many stops 
grouped_bystops = bus_data.groupby(['stop_id', 'stop_lat', 'stop_lon'])
numberstops = grouped_bystops['arrival_time'].nunique()
numberstops

stop_id  stop_lat   stop_lon  
1.0      41.392985  -81.536518     63
2.0      41.418571  -81.760947     72
4.0      41.503697  -81.618108    241
5.0      41.384506  -81.756317    108
14.0     41.498030  -81.692012    370
                                 ... 
50001.0  41.500290  -81.691917    492
60001.0  41.500764  -81.682781    172
60002.0  41.500256  -81.686436    166
60003.0  41.499918  -81.688906    166
60004.0  41.499515  -81.691994    161
Name: arrival_time, Length: 5799, dtype: int64

In [6]:
#numberstops.tolist()

In [7]:
stopnum_df = pd.DataFrame({"Number of Stops":numberstops})
stopnum_df.dtypes

Number of Stops    int64
dtype: object

In [8]:
# set index
stopnum_df.reset_index(inplace = True)

In [9]:
# Store lat and long
locations = stopnum_df[['stop_lat','stop_lon']]
stops = stopnum_df[['Number of Stops']].astype(float)

In [10]:
stop_sort = stopnum_df.sort_values(['Number of Stops'], ascending = True)
stop_sort

,stop_id,stop_lat,stop_lon,Number of Stops
445,1693.0,41.461686,-81.736618,1
5188,14209.0,41.404178,-81.467620,2
3850,9250.0,41.389785,-81.469126,2
4379,10658.0,41.398490,-81.468961,2
1852,5055.0,41.449909,-81.535955,2
...,...,...,...,...
5336,14412.0,41.501272,-81.692520,449
3729,8874.0,41.498977,-81.692950,457
3180,7694.0,41.497863,-81.697049,482
5794,50001.0,41.500290,-81.691917,492


Inserting of working population of the cities within the Cuhayoga county

In [11]:
cuhayoga_df = pd.read_csv('Resources/Cuhayoga_work_status.csv')
cuhayoga_census = cuhayoga_df
cuhayoga_census.head()

,GEO_ID,NAME,Unnamed: 2,Unnamed: 3,Population over 16 Years,DP03_0001M,DP03_0001PE,DP03_0001PM,In Labour Force,DP03_0002M,...,DP03_0135PE,DP03_0135PM,DP03_0136E,DP03_0136M,DP03_0136PE,DP03_0136PM,DP03_0137E,DP03_0137M,DP03_0137PE,DP03_0137PM
0,id,Geographic Area Name,NaN,NaN,Estimate!!EMPLOYMENT STATUS!!Population 16 yea...,Margin of Error!!EMPLOYMENT STATUS!!Population...,Percent Estimate!!EMPLOYMENT STATUS!!Populatio...,Percent Margin of Error!!EMPLOYMENT STATUS!!Po...,Estimate!!EMPLOYMENT STATUS!!Population 16 yea...,Margin of Error!!EMPLOYMENT STATUS!!Population...,...,Percent Estimate!!PERCENTAGE OF FAMILIES AND P...,Percent Margin of Error!!PERCENTAGE OF FAMILIE...,Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WH...,Margin of Error!!PERCENTAGE OF FAMILIES AND PE...,Percent Estimate!!PERCENTAGE OF FAMILIES AND P...,Percent Margin of Error!!PERCENTAGE OF FAMILIE...,Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WH...,Margin of Error!!PERCENTAGE OF FAMILIES AND PE...,Percent Estimate!!PERCENTAGE OF FAMILIES AND P...,Percent Margin of Error!!PERCENTAGE OF FAMILIE...
1,0600000US3903500000,County subdivisions not defined,Cuyahoga County,Ohio,0,11,0,(X),0,11,...,-,**,(X),(X),-,**,(X),(X),-,**
2,0600000US3903504416,Bay Village city,Cuyahoga County,Ohio,11912,174,11912,(X),8054,275,...,6.1,2.8,(X),(X),1.7,1,(X),(X),9.3,3.2
3,0600000US3903504500,Beachwood city,Cuyahoga County,Ohio,9521,243,9521,(X),5239,308,...,4.5,3.1,(X),(X),1.6,1.1,(X),(X),11.8,5.8
4,0600000US3903504878,Bedford city,Cuyahoga County,Ohio,10221,301,10221,(X),6641,461,...,7.1,3.3,(X),(X),10.4,5.3,(X),(X),14,4.6


In [12]:
cuhayoga_census = cuhayoga_census.rename(
    columns={"Unnamed: 3": "State"})
cuhayoga_census.head()

,GEO_ID,NAME,Unnamed: 2,State,Population over 16 Years,DP03_0001M,DP03_0001PE,DP03_0001PM,In Labour Force,DP03_0002M,...,DP03_0135PE,DP03_0135PM,DP03_0136E,DP03_0136M,DP03_0136PE,DP03_0136PM,DP03_0137E,DP03_0137M,DP03_0137PE,DP03_0137PM
0,id,Geographic Area Name,NaN,NaN,Estimate!!EMPLOYMENT STATUS!!Population 16 yea...,Margin of Error!!EMPLOYMENT STATUS!!Population...,Percent Estimate!!EMPLOYMENT STATUS!!Populatio...,Percent Margin of Error!!EMPLOYMENT STATUS!!Po...,Estimate!!EMPLOYMENT STATUS!!Population 16 yea...,Margin of Error!!EMPLOYMENT STATUS!!Population...,...,Percent Estimate!!PERCENTAGE OF FAMILIES AND P...,Percent Margin of Error!!PERCENTAGE OF FAMILIE...,Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WH...,Margin of Error!!PERCENTAGE OF FAMILIES AND PE...,Percent Estimate!!PERCENTAGE OF FAMILIES AND P...,Percent Margin of Error!!PERCENTAGE OF FAMILIE...,Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WH...,Margin of Error!!PERCENTAGE OF FAMILIES AND PE...,Percent Estimate!!PERCENTAGE OF FAMILIES AND P...,Percent Margin of Error!!PERCENTAGE OF FAMILIE...
1,0600000US3903500000,County subdivisions not defined,Cuyahoga County,Ohio,0,11,0,(X),0,11,...,-,**,(X),(X),-,**,(X),(X),-,**
2,0600000US3903504416,Bay Village city,Cuyahoga County,Ohio,11912,174,11912,(X),8054,275,...,6.1,2.8,(X),(X),1.7,1,(X),(X),9.3,3.2
3,0600000US3903504500,Beachwood city,Cuyahoga County,Ohio,9521,243,9521,(X),5239,308,...,4.5,3.1,(X),(X),1.6,1.1,(X),(X),11.8,5.8
4,0600000US3903504878,Bedford city,Cuyahoga County,Ohio,10221,301,10221,(X),6641,461,...,7.1,3.3,(X),(X),10.4,5.3,(X),(X),14,4.6


In [13]:
cuhayoga_rta = cuhayoga_census.loc[:, ["NAME", "State", "Population over 16 Years"]]
cuhayoga_rta.head()

,NAME,State,Population over 16 Years
0,Geographic Area Name,NaN,Estimate!!EMPLOYMENT STATUS!!Population 16 yea...
1,County subdivisions not defined,Ohio,0
2,Bay Village city,Ohio,11912
3,Beachwood city,Ohio,9521
4,Bedford city,Ohio,10221


In [14]:
# drop null rows
#no_null_crime_df = crime_df.dropna(how='any')
cuhayoga_cleaning = cuhayoga_rta.dropna(how='any')
cuhayoga_cleaning.head()

,NAME,State,Population over 16 Years
1,County subdivisions not defined,Ohio,0
2,Bay Village city,Ohio,11912
3,Beachwood city,Ohio,9521
4,Bedford city,Ohio,10221
5,Bedford Heights city,Ohio,9290


In [15]:
#add lat and lng columns for city
cuhayoga_cleaning['Lat'] = ''
cuhayoga_cleaning['Lng'] = ''
cuhayoga_cleaning.head()

,NAME,State,Population over 16 Years,Lat,Lng
1,County subdivisions not defined,Ohio,0,,
2,Bay Village city,Ohio,11912,,
3,Beachwood city,Ohio,9521,,
4,Bedford city,Ohio,10221,,
5,Bedford Heights city,Ohio,9290,,


In [16]:
# create a params dict that will be updated with new city each iteration
params = {"key": gkey}

# Loop through the cuhayoga_rta and run a lat/long search for each city in the county
for index, row in cuhayoga_cleaning.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    city = row['NAME']
    state = row['State']
    
    # update address key value
    params['address'] = f"{city},{state}"

    # make request
    cuhayoga_location = requests.get(base_url, params=params)
       
    # convert to json
    cuhayoga_location = cuhayoga_location.json()
    try:

        cuhayoga_cleaning.loc[index, "Lat"] = cuhayoga_location["results"][0]["geometry"]["location"]["lat"]
        cuhayoga_cleaning.loc[index, "Lng"] = cuhayoga_location["results"][0]["geometry"]["location"]["lng"]
    
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

# Visualize to confirm lat lng appear
cuhayoga_cleaning.head()

NameError: name 'gkey' is not defined

In [17]:
# Plot Heatmap

#Can we get weights to work???
fig_heat = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, dissipating=False, max_intensity=2, point_radius=.0025)
fig_heat.add_layer(heat_layer)
fig_heat

Figure(layout=FigureLayout(height='420px'))

In [18]:
# plot locations
fig_mark = gmaps.figure()
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig_mark.add_layer(markers)
fig_mark

Figure(layout=FigureLayout(height='420px'))

In [20]:
#visualize areas of interest
figure_layout = {
    'width': '800px',
    'height': '600px',
    'margin': '0 auto 0 auto'
}
fig_poly = gmaps.figure(layout=figure_layout,zoom_level=11, center=(41.45, -81.676029))
#fig_poly = gmaps.figure()
poly = gmaps.drawing_layer(features=[
    gmaps.heatmap_layer(locations, dissipating=False, max_intensity=2,                 point_radius=.0025),
    gmaps.Polygon(
        [(41.422357, -81.682310), (41.443596, -81.682455), (41.443533, -81.629649),         (41.421745, -81.636593), (41.410144, -81.648661), (41.413432, -81.683392)], fill_color=('blue')),
    gmaps.Polygon(
        [
            #burke
            (41.516709, -81.676029), 
            #lago e bank
            (41.501456, -81.705219),
            #flats
            (41.494781, -81.699536),
            #the jake (always the jake)
            (41.495057, -81.683945),
            #CSU
            (41.500102, -81.675685),
            #Slymans
            (41.513014, -81.671483)],
             fill_color=('green'))
        
        ])
fig_poly.add_layer(poly)

fig_poly
# add legends??

Figure(layout=FigureLayout(height='600px', margin='0 auto 0 auto', width='800px'))

Linking Population of cities to heatmap from current RTA stops.

In [21]:
#This is Leah's test line of code for merging

In [27]:
# Path to hospital data CSV file (includes names,coordinates)
hospitals_file = "Resources/hosp_data.csv"

# Read file to and display resulting dataframe
hospitals_df = pd.read_csv(hospitals_file)
hospitals_df.head()
hospitals_df

,hosp_name,hosp_lat,hosp_lon
0,Bedford Medical Center,41.395434,-81.534532
1,Cleveland Clinic Foundation,41.513757,-81.622297
2,Euclid Hospital,41.598870,-81.548521
3,Fairview Hospital,41.450264,-81.823845
4,Grace Hospital,41.480654,-81.690865
5,Hillcrest Hospital,41.519390,-81.434708
6,Huron Hospital,41.525826,-81.584320
7,Kaiser Permamente (Snow Road),41.404831,-81.775336
8,Kaiser Permanente (Cleveland Hts),41.514106,-81.550894
9,Lakewood Hospital,41.484856,-81.797217


In [28]:
###### Begin Hospital Code

### Simple plot of Hospital 
# Plot hospital locations
hfig_mark = gmaps.figure()
hlocations = hospitals_df[['hosp_lat','hosp_lon']]
# Marker variable
hosp_markers = gmaps.marker_layer(hlocations)
# Add map markers/display map
hfig_mark.add_layer(hosp_markers)
hfig_mark
###

Figure(layout=FigureLayout(height='420px'))

In [29]:
### Combination Map (Hospital + Bus Stop locations)
# Mock Original Heat Map
combo_heat = gmaps.figure()
new_layer = gmaps.heatmap_layer(locations, dissipating=False, max_intensity=2, point_radius=.0025)
combo_heat.add_layer(new_layer)

# Define hospital plots
newhosp_markers = gmaps.marker_layer(hlocations)

# Add map markers/display map
combo_heat.add_layer(newhosp_markers)

#Display Combination map 
combo_heat

###

######End Hospital Code

Figure(layout=FigureLayout(height='420px'))